# Tabular Record Compilation

In [5]:
from pathlib import Path
import os
import pandas as pd

base_dir = Path("../../dataset/clean/pre_production")
years = os.listdir(base_dir)

columns = ['Control Number', 'Cleaned Present Reading']
all_data = []

for year in years:
    files = os.listdir(base_dir / year)
    for file in files:
        month = file.split("_")[0]
        df = pd.read_csv(base_dir / year / file, usecols=columns)
        df['Month_Year'] = f"{year}_{month}"
        all_data.append(df)

# Combine into a single DataFrame
df_all = pd.concat(all_data, ignore_index=True)

# Pivot to wide format
wide_df = df_all.pivot_table(
    index=['Control Number'],
    columns='Month_Year',
    values='Cleaned Present Reading',
).reset_index()

wide_df = wide_df.sort_index(axis=1)

# Optional: Sort columns so months are in order
print(wide_df.head())


Month_Year  2019_DEC  2020_APR  2020_AUG  2020_DEC  2020_FEB  2020_JAN  \
0                NaN       NaN       NaN       NaN       NaN       NaN   
1                NaN       NaN       NaN       NaN       NaN       NaN   
2                NaN       NaN       NaN       NaN       NaN       NaN   
3                NaN    5465.0       NaN       NaN       NaN       NaN   
4                NaN    3280.0       NaN       NaN       NaN       NaN   

Month_Year  2020_JUL  2020_JUN  2020_MAR  2020_MAY  ...  2024_NOV  2024_OCT  \
0                NaN       NaN       NaN       NaN  ...       NaN       NaN   
1                NaN       NaN       NaN       NaN  ...       0.0       NaN   
2                NaN       NaN       NaN       NaN  ...       NaN       NaN   
3                NaN       NaN       NaN       NaN  ...       0.0       0.0   
4                NaN       NaN       NaN       NaN  ...    2629.0    2609.0   

Month_Year  2024_SEP  2025_APR  2025_FEB  2025_JAN  2025_MAR  2025_MAY  \
0     

In [42]:
%pip install calendar

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement calendar (from versions: none)

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for calendar


In [6]:
# ================================================
# Consolidate Wide Data with Connection Status and Last Reading
# ================================================

import calendar
import os
from utilities import read_csv, save_csv

# ------------------------------------------------
# 1. Read source datasets
# ------------------------------------------------
right_df = read_csv("../../dataset/clean/pre_production/2025/SEP_2025_pre_production.csv")  
name_df = read_csv("../../dataset/clean/semi_clean/cleaned_master.csv") 

# ------------------------------------------------
# 2. Merge connection status from pre-production
# ------------------------------------------------
final_df = wide_df.merge(
    right_df[['Control Number', 'Connection Status']],
    on='Control Number',
    how='left'
)

# Normalize connection status values
final_df['Connection Status'] = (
    final_df['Connection Status']
    .replace({"Invalid Reading": "Active"})  # Convert "Invalid Reading" to "Active"
    .fillna("Unknown")                       # Fill missing with "Unknown"
)

# ------------------------------------------------
# 3. Identify month columns (exclude non-monthly fields)
# ------------------------------------------------
fixed_cols_pre_sort = [col for col in final_df.columns if not col[0].isdigit()]  # Columns that are not YYYY_MON format
month_cols = [col for col in final_df.columns if col not in fixed_cols_pre_sort]

# ------------------------------------------------
# 4. Sort month columns chronologically
# ------------------------------------------------
month_map = {m.upper(): i for i, m in enumerate(calendar.month_abbr) if m}  # {"JAN":1, "FEB":2, ...}

month_cols_sorted = sorted(
    month_cols,
    key=lambda x: (int(x.split('_')[0]), month_map[x.split('_')[1].upper()])
)

# ------------------------------------------------
# 5. Compute the "Last Reading" (latest non-null value per row)
# ------------------------------------------------
final_df["Last Reading"] = final_df[month_cols_sorted].apply(
    lambda row: row.dropna().iloc[-1] if row.dropna().size > 0 else None,
    axis=1
)

# ------------------------------------------------
# 6. Merge account names from master list
# ------------------------------------------------
final_df = final_df.merge(
    name_df[['Control Number', 'Account Name']],
    on='Control Number',
    how='left'
)

# ------------------------------------------------
# 7. Final column ordering: fixed fields first, then months
# ------------------------------------------------
fixed_cols_final = ['Control Number', 'Account Name', 'Connection Status', 'Last Reading']
final_df = final_df[fixed_cols_final + month_cols_sorted]

# ------------------------------------------------
# 8. Save the consolidated output
# ------------------------------------------------
save_path = "../../dataset/clean/production/tabular/consolidated.csv"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
save_csv(final_df, save_path)

# ------------------------------------------------
# 9. Preview output for sanity check
# ------------------------------------------------
display(final_df.head())
display(final_df.info())


,Control Number,Account Name,Connection Status,Last Reading,2019_DEC,2020_JAN,2020_FEB,2020_MAR,2020_APR,2020_MAY,...,2024_SEP,2024_OCT,2024_NOV,2024_DEC,2025_JAN,2025_FEB,2025_MAR,2025_APR,2025_MAY,2025_SEP
0,0.0,Kho Shipping Lines,Unknown,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,121.0,"Almodal, Maricel",Disconnected,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50964.0,"Esparraguerra, Irish",Unknown,129.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,500001.0,"Encinares, Roberto",Disconnected,0.0,NaN,NaN,NaN,NaN,5465.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,500002.0,"Pareja, Myrna",Active,2782.0,NaN,NaN,NaN,NaN,3280.0,NaN,...,NaN,2609.0,2629.0,2629.0,2629.0,NaN,NaN,NaN,2782.0,2782.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2175 entries, 0 to 2174
Data columns (total 59 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Control Number     2175 non-null   float64
 1   Account Name       2074 non-null   object 
 2   Connection Status  2175 non-null   object 
 3   Last Reading       2175 non-null   float64
 4   2019_DEC           1142 non-null   float64
 5   2020_JAN           1175 non-null   float64
 6   2020_FEB           1207 non-null   float64
 7   2020_MAR           1227 non-null   float64
 8   2020_APR           1619 non-null   float64
 9   2020_MAY           1239 non-null   float64
 10  2020_JUN           1233 non-null   float64
 11  2020_JUL           1261 non-null   float64
 12  2020_AUG           1293 non-null   float64
 13  2020_SEP           1268 non-null   float64
 14  2020_OCT           1271 non-null   float64
 15  2020_NOV           1272 non-null   float64
 16  2020_DEC           1142 

None

In [8]:
# Create csv for disconnected accounts
disconnected_df = final_df[(final_df['Connection Status'] == 'Disconnected') | (final_df['Connection Status'] == 'Unknown')]
disconnected_df.to_csv("../../dataset/clean/production/tabular/disconnected.csv", index=False)
